# Задача о разборчивой невесте

* [Описание на wikipedia](https://ru.wikipedia.org/wiki/%D0%97%D0%B0%D0%B4%D0%B0%D1%87%D0%B0_%D0%BE_%D1%80%D0%B0%D0%B7%D0%B1%D0%BE%D1%80%D1%87%D0%B8%D0%B2%D0%BE%D0%B9_%D0%BD%D0%B5%D0%B2%D0%B5%D1%81%D1%82%D0%B5)
* [Брошюра Гусейн-Заде](https://www.mccme.ru/mmmf-lectures/books/books/book.25.pdf)

Задача: провести статистические испытания разных стратегий поведения невест. Следует заполнить недостающий код. Примеры выводов, которые необходимо получить, даны в конце блокнота.

In [1]:
class Bride:
    """
    Абстрактная невеста. Базовый класс для всех стратегий поведения невест.
    
    Соглашение о женихах:
    Женихи - целые числа от 0 до n.
    0 -- самый привлекательный жених, (n-1) -- самый нежеланный.
    Можно считать, что этот номер -- это место в "забеге среди женихов".
    Проводя дальнейшую аналогию, можно сказать, что
    0, 1 и 2 -- это золото, серебро и бронза.
    
    Соглашение о невесте:
    Для статистических испытаний создаём одну невесту, а между разными сессиями
    по подбору жениха стираем ей память методом clear.
    
    Метод match(self, groom) -- это один шаг в сессии подбора жениха.
    Возвращает либо True (подходит), либо False (не подходит).
    
    Метод clear(self) -- очищает память невесты, чтобы можно было начать
    новую сессию по подбору.
    """
    def match(self, groom):
        raise NotImplementedError('abstract method')
    
    def clear(self):
        raise NotImplementedError('abstract method')

In [37]:
class ImpatientBride(Bride):
    """
    Стратегия поведения нетерпеливой невесты. Соглашается на первого встречного.
    """
    def match(self, groom):
        return True
    
    def clear(self):
        pass

In [38]:
class ForgetfulBride(Bride):
    """
    Стратегия поведения забывчивой невесты. Соглашается, если текущий жених лучше
    нескольких предыдущих.
    """
    def __init__(self, memory_size=3):
        """
        Parameters
        ----------
        memory_size
          Размер памяти забывчивой невесты. По-умолчанию, помнит предыдущих трёх.
        """
        self.memory_size = memory_size
        self.memory = []
    
    def match(self, groom):
        if len(self.memory) < self.memory_size:
            # Недостаточно "опыта". Посмотрели и отказали.
            self.memory.append(groom)
            return False
        elif groom >= min(self.memory):
            # Недостаточно хороший жених. Хуже предыдущих.
            self.memory.append(groom)
            self.memory.pop(0)
            return False
        else:
            # Достаточно хороший жених.
            return True
    
    def clear(self):
        self.memory.clear()

In [42]:
import math

class SelectiveBride(Bride):
    """
    Стратегия поведения разборчивой невесты. Первые 1/e женихов пропускает, набирая опыта.
    Потом соглашается на первого, кто будет лучшим. См. брошюру Гусейн-Заде.
    """
    def __init__(self, size, threshold=None):
        """
        Parameters
        ----------
        size: int
          Количество женихов.
        
        threshold: int
          Порог обучения, после которого невеста начинает выбирать лучшего.
        """
        raise NotImplementedError('надо реализовать')
    
    def match(self, groom):
        raise NotImplementedError('надо реализовать')
    
    def clear(self):
        raise NotImplementedError('надо реализовать')

In [48]:
import random


class MatchingSessions:
    """
    Проведение статистических испытаний стратегий поведения невесты.
    """
    def __init__(self, bride, grooms=1000, sessions=10000):
        """
        Parameters
        ----------
        bride
          Невеста для испытаний.
        
        grooms
          Либо список женихов, либо их количество.
        
        sessions
          Количество сессий статистических испытаний.
        """
        self.bride = bride
        
        self.grooms = []
        if isinstance(grooms, int):
            self.grooms = list(range(grooms))
        elif isinstance(grooms, list):
            self.grooms = grooms
        else:
            msg = 'improper grooms type {}'.format(type(grooms))
            raise ValueError(msg)
        
        self.sessions_number = sessions

    def run_single_session(self):
        """
        Проведение одной сессии по подбору женихов. 

        Returns
        -------
        Подобранный жених. Либо последний, если невеста не дала согласия.
        """
        raise NotImplementedError('надо реализовать, использовать random.shuffle для перемешивания женихов')

    def run(self):
        """
        Проведение серии статистических испытаний. Вывод статистики.
        
        Returns
        -------
        Словарь с посчитанной статистикой:
          * mean -- средний номер выбранного жениха,
          * best -- вероятность выбора наилучшего жениха (под номером 0),
          * medalists -- вероятность выбора жениха из первой тройки (под номерами 0, 1 или 2).
        """
        raise NotImplementedError('надо реализовать')

# Примеры выводов

In [47]:
# Статистики забывчивой невесты, помнящей только трёх последних женихов.
MatchingSessions(ForgetfulBride(3)).run()

{'mean': 221.6468, 'best': 0.0037, 'medalists': 0.0099}

In [36]:
# Статистики нетерпеливой невесты, выбирающей первого попавшегося.
MatchingSessions(ImpatientBride()).run()

{'mean': 498.0663, 'best': 0.0008, 'medalists': 0.0028}

In [43]:
# Статистики стандартной разборчивой невесты.
MatchingSessions(SelectiveBride(1000)).run()

{'mean': 182.1176, 'best': 0.3677, 'medalists': 0.5696}

In [44]:
# Статистики разборчивой невесты с обучением на 20% женихов.
MatchingSessions(SelectiveBride(1000, 200)).run()

{'mean': 103.1865, 'best': 0.316, 'medalists': 0.5797}

In [46]:
# Статистики разборчивой невесты с обучением на 50% женихов.
MatchingSessions(SelectiveBride(1000, 500)).run()

{'mean': 245.0093, 'best': 0.3532, 'medalists': 0.4836}